In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

# Overview of the data 

In [2]:
%%sql

SELECT
    name,
    type    
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Selecting Albums to Purchase 

The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

|      Artist Name     |  Genre  |
|:--------------------:|:-------:|
| Regal                | Hip-Hop |
| Red Tone             | Punk    |
| Meteor and the Girls | Pop     |
| Slim Jim Bites       | Blues   |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA and make a recommendation for the three artists whose albums we should purchase for the store.

In [3]:
%%sql

WITH us_customer AS (
                     SELECT
                         customer_id,
                         country
                     FROM customer
                     WHERE country = 'USA'                    
                    ),
     us_purchases AS (
                      SELECT
                          i.invoice_id
                      FROM invoice AS i
                      INNER JOIN us_customer AS usc ON i.customer_id = usc.customer_id
                     ),
     us_detailed_purchases AS (
                               SELECT
                                   il.invoice_line_id,
                                   il.invoice_id,
                                   il.track_id,
                                   il.quantity
                               FROM us_purchases AS usp
                               INNER JOIN invoice_line AS il ON usp.invoice_id = il.invoice_id                             
                              ),
     track_genre AS (
                     SELECT
                         t.track_id,
                         g.name AS genre
                     FROM track AS t
                     INNER JOIN genre AS g ON t.genre_id = g.genre_id         
                    ),
     genre_sales AS (
                     SELECT 
                         tg.genre,
                         SUM(quantity) AS number_of_tracks_sold
                     FROM us_detailed_purchases AS usdp
                     INNER JOIN track_genre AS tg ON usdp.track_id = tg.track_id
                     GROUP BY tg.genre
                     ORDER BY number_of_tracks_sold DESC
                    ),
     total_tracks_sold AS (
                           SELECT SUM(number_of_tracks_sold) AS sum_total
                           FROM genre_sales        
                          ),
     genre_sales_overview AS (
                              SELECT
                                  genre,
                                  number_of_tracks_sold AS 'Number of tracks sold',
                                  CAST(number_of_tracks_sold AS FLOAT) / sum_total AS 'Number of tracks sold (in %)'
                              FROM genre_sales, total_tracks_sold                                 
                             ),
     considered_genres AS (
                           SELECT *
                           FROM genre_sales_overview
                           WHERE genre LIKE "%Hip%"
                           OR genre LIKE "%Punk%"
                           OR genre LIKE "%Pop%"
                           OR genre LIKE "%Blues%"                             
                          )
SELECT genre AS "Top 3 Genres"
FROM considered_genres
LIMIT 3;


 * sqlite:///chinook.db
Done.


Top 3 Genres
Alternative & Punk
Blues
Pop


* The top 3 genres by sales amongst the possible genres to be introduced are 'Punk', 'Blues', and 'Pop'.
* Hence choose albums produced by the artists Red Tone, Slim Jim Bites, and Meteor and the Girls

# Analyzing Employee Sales Performance 

* Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase.
* We have to analyze the purchases of customers belonging to each employee (i.e. we have to calculate the total dollar amount of sales assigned to each sales support agent within the company) to see if any sales support agent is performing either better or worse than the others.

In [4]:
%%sql

WITH customer_total_purchased AS (
                                  SELECT
                                      customer_id,
                                      SUM(total) AS total_purchases
                                  FROM invoice
                                  GROUP BY customer_id                        
                                 ),
     sales_support_agent AS (
                             SELECT *
                             FROM employee
                             WHERE title = 'Sales Support Agent'                 
                            ),
     sales_support_agent_customers AS (
                                       SELECT 
                                           ssa.*,
                                           c.customer_id
                                       FROM sales_support_agent AS ssa
                                       LEFT JOIN customer AS c ON ssa.employee_id = c.support_rep_id     
                                      ),
     total_agent_sales AS (
                           SELECT
                               ssac.employee_id,
                               ssac.first_name || ' ' || ssac.last_name AS name,
                               ssac.hire_date,
                               SUM(ctp.total_purchases) AS total_sales
                           FROM sales_support_agent_customers AS ssac
                           LEFT JOIN customer_total_purchased AS ctp ON ssac.customer_id = ctp.customer_id
                           GROUP BY ssac.employee_id                     
                          )
SELECT *
FROM total_agent_sales;

 * sqlite:///chinook.db
Done.


employee_id,name,hire_date,total_sales
3,Jane Peacock,2017-04-01 00:00:00,1731.5099999999998
4,Margaret Park,2017-05-03 00:00:00,1584.0000000000002
5,Steve Johnson,2017-10-17 00:00:00,1393.92


* By looking only at the numbers, we might conclude that Jane performs better than Margaret, and Margaret performs better than Steve.
* But by having a look at the hire date, we see that employees who joined earlier have cumulative sales amount more than that for those who joined later.

# Analyzing Sales by Country 

In [5]:
%%sql

WITH invoice_customer AS (
                          SELECT
                              i.invoice_id,
                              i.customer_id,
                              i.total AS order_total,
                              c.country AS customer_country
                          FROM invoice AS i
                          LEFT JOIN customer AS c ON i.customer_id = c.customer_id                            
                         ),
     country_mapping AS (
                         SELECT
                             ic.customer_country,
                             CASE
                                 WHEN COUNT(DISTINCT(ic.customer_id)) = 1 THEN "Other"
                                 ELSE ic.customer_country
                             END
                             AS country_group
                         FROM invoice_customer AS ic
                         GROUP BY ic.customer_country                      
                        ),
     country_sales_analysis AS (
                                SELECT
                                    cmap.country_group AS country_of_customer,
                                    COUNT(DISTINCT(ic.customer_id)) AS total_number_of_customers,
                                    SUM(ic.order_total) AS total_value_of_sales,
                                    SUM(ic.order_total) / COUNT(DISTINCT(ic.customer_id)) AS average_value_of_sales_per_customer,
                                    SUM(ic.order_total) / COUNT(ic.invoice_id) AS average_order_value,
                                    CASE
                                        WHEN cmap.country_group = 'Other' THEN 1
                                        ELSE 0
                                    END
                                    AS sorting_code
                                FROM invoice_customer AS ic
                                INNER JOIN country_mapping AS cmap ON ic.customer_country = cmap.customer_country
                                GROUP BY cmap.country_group
                                ORDER BY total_value_of_sales DESC
                               ),
     formatted_country_sales_analysis AS (
                                          SELECT
                                              csa.country_of_customer,
                                              csa.total_number_of_customers,
                                              csa.total_value_of_sales,
                                              csa.average_value_of_sales_per_customer,
                                              csa.average_order_value
                                          FROM country_sales_analysis AS csa
                                          ORDER BY csa.sorting_code                                         
                                         )
SELECT *
FROM formatted_country_sales_analysis;

 * sqlite:///chinook.db
Done.


country_of_customer,total_number_of_customers,total_value_of_sales,average_value_of_sales_per_customer,average_order_value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143


# Albums vs Individual Tracks 

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

1. purchase a whole album
2. purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

We have two edge cases to consider:

1. Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
2. Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

In [6]:
%%sql

WITH invoices_overview AS (
                           SELECT
                               il.invoice_id,
                               il.track_id
                           FROM invoice_line AS il
                          ),
     tracks_overview AS (
                         SELECT
                             t.track_id,
                             t.name AS track_name,
                             t.album_id
                         FROM track AS t     
                        ),
     invoice_details AS (
                         SELECT
                             iv.invoice_id,
                             iv.track_id,
                             tk.track_name,
                             tk.album_id
                         FROM invoices_overview AS iv
                         INNER JOIN tracks_overview AS tk ON iv.track_id = tk.track_id     
                        ),
     purchase_type_mapper AS (
                              SELECT
                                  ivdet.invoice_id,
                                  CASE
                                      WHEN
                                          (
                                           SELECT iv.track_id
                                           FROM invoices_overview AS iv
                                           WHERE iv.invoice_id = ivdet.invoice_id
                                          
                                           EXCEPT
                                           
                                           SELECT tk.track_id
                                           FROM tracks_overview AS tk
                                           WHERE tk.album_id = ivdet.album_id
                                          ) IS NULL
                                          
                                          AND
                                          
                                          (
                                           SELECT tk.track_id
                                           FROM tracks_overview AS tk
                                           WHERE tk.album_id = ivdet.album_id
                                              
                                           EXCEPT
                                           
                                           SELECT iv.track_id
                                           FROM invoices_overview AS iv
                                           WHERE iv.invoice_id = ivdet.invoice_id
                                          ) IS NULL
                                      THEN "album"
                                      ELSE "collection"
                                  END
                                  AS purchasing_strategy
                              FROM invoice_details AS ivdet
                              GROUP BY ivdet.invoice_id     
                             ),
     edge_case_mapper AS (
                          SELECT
                              ivdet.invoice_id,
                              ptmap.purchasing_strategy,
                              CASE
                                  WHEN 
                                      ptmap.purchasing_strategy = 'album'
                                      AND
                                      COUNT(ivdet.track_id) < 3
                                  THEN "yes"
                                  WHEN
                                      ptmap.purchasing_strategy = 'album'
                                      AND
                                      COUNT(ivdet.track_id) >= 3
                                  THEN "no"
                                  WHEN 
                                      ptmap.purchasing_strategy = 'collection'
                                      AND
                                      (
                                          SELECT
                                              idet.album_id || COUNT(idet.track_id)
                                          FROM invoice_details AS idet
                                          WHERE idet.invoice_id = ivdet.invoice_id
                                          GROUP BY idet.album_id
                                          
                                          INTERSECT
                                          
                                          SELECT
                                              tk.album_id || COUNT(tk.track_id)
                                          FROM tracks_overview AS tk
                                          WHERE tk.album_id IN (
                                                                SELECT DISTINCT(idet.album_id)
                                                                FROM invoice_details AS idet
                                                                WHERE idet.invoice_id = ivdet.invoice_id
                                                               )
                                          GROUP BY tk.album_id            
                                      ) IS NOT NULL
                                  THEN "yes"
                                  ELSE "no" 
                              END
                              AS edge_case
                          FROM invoice_details AS ivdet
                          INNER JOIN purchase_type_mapper AS ptmap ON ivdet.invoice_id = ptmap.invoice_id 
                          GROUP BY ivdet.invoice_id        
                         )
SELECT
    edgemap.purchasing_strategy AS "Purchasing Strategy",
    COUNT(i.invoice_id) AS "Number of Invoices",
    COUNT(i.invoice_id) / CAST((SELECT COUNT(invoice_id) FROM invoice) AS FLOAT) AS "Percentage of Invoices"
FROM invoice AS i
INNER JOIN edge_case_mapper AS edgemap ON i.invoice_id = edgemap.invoice_id
GROUP BY edgemap.purchasing_strategy;

 * sqlite:///chinook.db
Done.


Purchasing Strategy,Number of Invoices,Percentage of Invoices
album,114,0.18566775244299674
collection,500,0.8143322475570033


* The Chinook store should not continue to buy full albums from record companies.

# Artist who is used in the most playlists 

In [7]:
%%sql

WITH track_artist_overview AS (
                               SELECT
                                   t.track_id,
                                   al.artist_id
                               FROM track AS t
                               INNER JOIN album AS al ON t.album_id = al.album_id
                              ),
     artist_playlist_occurances AS (
                                    SELECT
                                        tao.artist_id,
                                        COUNT(DISTINCT(pt.playlist_id)) AS number_of_playlists_occured_in
                                    FROM playlist_track AS pt
                                    INNER JOIN track_artist_overview AS tao ON pt.track_id = tao.track_id
                                    GROUP BY tao.artist_id
                                    ORDER BY number_of_playlists_occured_in DESC
                                   )
SELECT
    a.name AS artist_name,
    apo.number_of_playlists_occured_in
FROM artist_playlist_occurances AS apo
INNER JOIN artist AS a ON apo.artist_id = a.artist_id
;

 * sqlite:///chinook.db
Done.


artist_name,number_of_playlists_occured_in
Eugene Ormandy,7
Berliner Philharmoniker & Herbert Von Karajan,6
The King's Singers,6
Academy of St. Martin in the Fields & Sir Neville Marriner,6
English Concert & Trevor Pinnock,6
Philip Glass Ensemble,5
"Mela Tenenbaum, Pro Musica Prague & Richard Kapp",5
Martin Roscoe,5
Julian Bream,5
Kent Nagano and Orchestre de l'Opéra de Lyon,5


* The artist Eugene Ormandy, appeared in the most playlists (7 times).

# Tracks that have been purchased vs not purchased 

In [8]:
%%sql

SELECT
    COUNT(DISTINCT(track.track_id)) AS 'Tracks Available',
    COUNT(DISTINCT(invoice_line.track_id)) AS 'Tracks Purchased',
    COUNT(DISTINCT(track.track_id)) - COUNT(DISTINCT(invoice_line.track_id)) AS 'Tracks not Purchased'
FROM track,invoice_line;

 * sqlite:///chinook.db
Done.


Tracks Available,Tracks Purchased,Tracks not Purchased
3503,1806,1697


# Q : Is the range of tracks in the store reflective of their sales popularity? 

In [9]:
%%sql

SELECT COUNT(*) AS 'Number Of Tracks in Store' 
FROM track
;

 * sqlite:///chinook.db
Done.


Number Of Tracks in Store
3503


In [10]:
%%sql

WITH store_tracks AS (
                      SELECT track_id
                      FROM track AS t        
                     ),
     invoice_tracks AS (
                        SELECT
                            track_id,
                            quantity AS track_quantity
                        FROM invoice_line AS il     
                       )
SELECT
    st.track_id,
    COUNT(it.track_quantity) AS Number_Of_Track_Purchases
FROM store_tracks AS st
LEFT JOIN invoice_tracks AS it ON st.track_id = it.track_id
GROUP BY st.track_id;

 * sqlite:///chinook.db
Done.


track_id,Number_Of_Track_Purchases
1,8
2,3
3,1
4,4
5,1
6,13
7,4
8,6
9,9
10,6


* The range of tracks in the store is not reflective of their sales popularity.

# Effect of media type on popularity of tracks bought 

In [11]:
%%sql

CREATE VIEW invoice_track_protection AS
    WITH media_type_protection AS (
                                   SELECT
                                       media_type_id,
                                       CASE
                                           WHEN name LIKE '%protected%' THEN 'protected'
                                           ELSE 'non-protected'
                                       END
                                       AS protection_offered
                                   FROM media_type
                                  ),
         track_media_type AS (
                              SELECT
                                  track_id,
                                  media_type_id
                              FROM track     
                             ),
         track_protection AS (
                              SELECT
                                  tmt.track_id,
                                  mtp.protection_offered
                              FROM track_media_type AS tmt
                              INNER JOIN media_type_protection AS mtp ON tmt.media_type_id = mtp.media_type_id     
                             ),
         invoice_track_quantity AS (
                                    SELECT
                                        track_id,
                                        quantity
                                    FROM invoice_line     
                                   )
    SELECT
        itq.track_id,
        itq.quantity,
        tp.protection_offered
    FROM invoice_track_quantity AS itq
    LEFT JOIN track_protection AS tp ON itq.track_id = tp.track_id;

 * sqlite:///chinook.db
Done.


[]

## Overview

In [12]:
%%sql 

SELECT
    protection_offered,
    SUM(quantity) AS total_track_sales
FROM invoice_track_protection
GROUP BY protection_offered;

 * sqlite:///chinook.db
Done.


protection_offered,total_track_sales
non-protected,4315
protected,442


* tracks having protections on them are sold much lesser (approx. 1/10th of non-protected tracks) than their unprotected tracks.

## media types of popular tracks (according to sales info)

In [13]:
%%sql

SELECT
    track_id,
    SUM(quantity) AS track_sales_volume,
    protection_offered
FROM invoice_track_protection
GROUP BY track_id
ORDER BY track_sales_volume DESC;

 * sqlite:///chinook.db
Done.


track_id,track_sales_volume,protection_offered
3336,31,non-protected
1489,14,non-protected
1495,14,non-protected
6,13,non-protected
1487,13,non-protected
1490,13,non-protected
1483,12,non-protected
2558,12,non-protected
1129,11,non-protected
1479,11,non-protected


* As can be seen, amongst the top 20 tracks, there are no tracks that are protected. Hence, non-protected tracks are more popular amongst customers.